In [1]:
import os 
import sys
import tensorflow as tf
import numpy as np
import pickle
from tensorflow.keras.models import load_model

sys.path.append("../")

from src.data.make_dataset import INDEX_SHEET_NAME,load_data
from src.features.build_features import generate_features
from src.models.LSTM import build_lstm_model, generate_train_val_data
from src.models.metrics import LinearCorrelation, MeanAbsolutePercentageError, TheilU

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
INDEX_SHEET_NAME

['HangSeng Index Data',
 'S&P500 Index Data',
 'CSI300 Index Data',
 'DJIA index Data',
 'Nikkei 225 index Data',
 'Nifty 50 index Data']

In [3]:
raw = load_data(sheet_name=INDEX_SHEET_NAME[-1])
raw.head()

Start downloading data...
Download complete.


,Date,Ntime,Open Price,High Price,Low price,Closing Price,Volume,MACD,CCI,ATR,...,EMA20,MA5,MA10,MTM6,MTM12,ROC,SMI,WVAD,US dollar Index,Interbank Offered Rate
0,20080701,733590,4039.75,4075.40,3878.20,3896.75,164469220,-201.56,-180.03,197.2,...,4385.058946,4128.49,4253.415,-369.65,-620.35,-13.733369,-0.149627,-828023961.0,72.34,8.706
1,20080702,733591,3895.30,4107.15,3848.25,4093.35,199920144,-199.63,-131.91,258.9,...,4357.277142,4096.63,4204.510,-97.75,-479.15,-10.478950,-0.091225,-561033032.0,71.99,7.730
2,20080703,733592,4094.60,4097.35,3874.85,3925.75,154573765,-209.21,-129.55,222.5,...,4316.179319,4018.61,4146.660,-326.90,-727.25,-15.629701,-0.097020,-726253640.3,72.73,6.400
3,20080704,733593,3926.65,4033.50,3896.40,4016.00,152045352,-207.14,-111.08,137.1,...,4287.590812,3994.48,4113.505,-299.85,-566.40,-12.360335,-0.058691,-548766472.1,72.71,6.210
4,20080707,733596,4002.00,4114.50,4002.00,4030.00,125737237,-202.03,-80.86,112.5,...,4263.058354,3992.37,4089.865,-106.65,-474.25,-10.528945,-0.038439,-486195115.1,72.71,9.000


### WSAE-LSTM

In [ ]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME[1:]:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/wsae/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+'/wase-lstmt_config.json') \
            or not os.path.exists(model_save_dir+'wase-lstm_weights.h5'):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+'/wase-lstmt_config.json', 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+'/wase-lstm_weights.h5')
            print("Model Saved!")
        else:
            with open(model_save_dir+'/wase-lstmt_config.json') as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=0.05
                              )
            lstm.load_weights(model_save_dir+'wase-lstm_weights.h5')

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./WSAE-LSTM/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./WSAE-LSTM/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start S&P500 Index Data part!
>>>>201010 finished!<<<<
>>>>201101 finished!<<<<
>>>>201104 finished!<<<<
>>>>201107 finished!<<<<
>>>>201110 finished!<<<<
>>>>201201 finished!<<<<
>>>>201204 finished!<<<<
>>>>201207 finished!<<<<
>>>>201210 finished!<<<<
>>>>201301 finished!<<<<
>>>>201304 finished!<<<<
>>>>201307 finished!<<<<
>>>>201310 finished!<<<<
>>>>201401 finished!<<<<
>>>>201404 finished!<<<<
>>>>201407 finished!<<<<
>>>>201410 finished!<<<<
>>>>201501 finished!<<<<
>>>>201504 finished!<<<<
>>>>201507 finished!<<<<
>>>>201510 finished!<<<<
>>>>201601 finished!<<<<
>>>>201604 finished!<<<<
>>>>201607 finished!<<<<
>>>> Feature generation complete! <<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 11ms/step - loss: 5.8790e-04 - mean_absolute_percentage_error: 2.2837 - mape: 0.0228 - r: 0.8632 - theil_u: 0.0143
>>>>S&P500 Index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 

In [ ]:
import pickle
with open(f'train_result.pickle', 'rb') as handle:
    result_dict=pickle.load(handle)

In [11]:
raw=load_data(sheet_name='CSI300 Index Data')

In [12]:
raw.head()

,Ntime,Time,Closing Price,Open Price,High price,Low Price,Volume,MACD,CCI,ATR,...,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,HIBOR
0,20080702,733591,21704.4492,21785.3906,21938.1992,21555.5293,254858.26,-646.053200,-193.115118,546.4805,...,22956.709411,22497.96286,-751.5703,22187.92768,-887.8516,-5.754486,-0.069509,-401619.369837,71.99,1.6
1,20080703,733592,21242.7793,21389.4902,21742.0703,21163.5703,272528.48,-702.282620,-186.955957,578.5000,...,22793.477972,22289.66071,-1392.3806,21909.45156,-1786.9102,-7.872373,-0.068201,-507305.257079,72.73,1.6
2,20080704,733593,21423.8203,21402.1699,21534.0508,21344.8496,179634.42,-723.891770,-133.742667,291.2715,...,22663.034384,22152.28180,-1031.8496,21703.08164,-1634.1699,-8.153977,-0.058111,-398848.614015,72.71,1.6
3,20080707,733596,21913.0605,21402.6992,21916.2109,21402.6992,196457.92,-693.544830,-77.445777,513.5117,...,22591.608300,22069.02789,-129.2891,21677.22382,-1412.7403,-3.880007,-0.035303,-264228.169678,72.71,1.6
4,20080708,733597,21220.8105,21632.6992,21684.2109,21098.8398,201519.68,-717.087314,-108.634123,814.2207,...,22461.056128,21919.61285,-881.1993,21500.98396,-1576.7989,-6.703666,-0.055006,-518512.844172,72.96,1.6


In [ ]:
result_dict

### WLSTM

In [6]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/wavelet/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/wlstmt_config.json'
        weight_filename = 'wlstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+config_filename) \
            or not os.path.exists(model_save_dir+weight_filename):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 19],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=learning_rate
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./WLSTM/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./WLSTM/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start HangSeng Index Data part!
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 13ms/step - loss: 0.0068 - mean_absolute_percentage_error: 6.9749 - mape: 0.0697 - r: 0.5543 - theil_u: 0.0416
>>>>HangSeng Index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 15ms/step - loss: 0.0029 - mean_absolute_percentage_error: 4.2920 - mape: 0.0429 - r: 0.7295 - theil_u: 0.0267
>>>>HangSeng Index Data 201101 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 17ms/step - loss: 0.0015 - mean_absolute_percentage_error: 3.7271 - mape: 0.0373 - r: 0.8168 - theil_u: 0.0230
>>>>HangSeng Index Data 201104 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.0121 - mean_absolute_percentage_error: 16.2649 - mape: 0.1626 - r: 0.6098 - theil_u: 0.0876
>>>>HangSeng Index Data 201107 done!<<<<

Model Saved!
1/1 [==============================] - 0s 17ms/step - loss: 0.0016 - mean_absolute_percentage_error: 3.2091 - mape: 0.0321 - r: 0.9443 - theil_u: 0.0207
>>>>S&P500 Index Data 201207 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 19ms/step - loss: 0.0017 - mean_absolute_percentage_error: 3.3451 - mape: 0.0335 - r: 0.7454 - theil_u: 0.0210
>>>>S&P500 Index Data 201210 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 15ms/step - loss: 0.0037 - mean_absolute_percentage_error: 4.1089 - mape: 0.0411 - r: 0.8307 - theil_u: 0.0269
>>>>S&P500 Index Data 201301 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 15ms/step - loss: 0.0898 - mean_absolute_percentage_error: 20.3377 - mape: 0.2034 - r: 0.5013 - theil_u: 0.1190
>>>>S&P500 Index Data 201304 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [=====================

>>>>CSI300 Index Data 201404 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 16ms/step - loss: 0.0076 - mean_absolute_percentage_error: 28.4077 - mape: 0.2841 - r: 0.8397 - theil_u: 0.1193
>>>>CSI300 Index Data 201407 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 17ms/step - loss: 0.2262 - mean_absolute_percentage_error: 28.1494 - mape: 0.2815 - r: 0.9347 - theil_u: 0.2550
>>>>CSI300 Index Data 201410 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 19ms/step - loss: 1.4201 - mean_absolute_percentage_error: 54.5748 - mape: 0.5457 - r: 0.6379 - theil_u: 0.3818
>>>>CSI300 Index Data 201501 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.8195 - mean_absolute_percentage_error: 45.3430 - mape: 0.4534 - r: 0.0457 - theil_u: 0.3068
>>>>CSI300 Index Data 201504 done!<<

C:\Users\mrliu\AppData\Local\Continuum\anaconda3\envs\envTF20\lib\site-packages\pywt\_thresholding.py:54: RuntimeWarning: invalid value encountered in less
  cond = np.less(np.absolute(data), value)


>>>>201110 finished!<<<<
>>>>201201 finished!<<<<


..\src\features\build_features.py:24: RuntimeWarning: invalid value encountered in subtract
  return (x - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))
..\src\features\build_features.py:24: RuntimeWarning: invalid value encountered in true_divide
  return (x - x_train.min(axis=0)) / (x_train.max(axis=0) - x_train.min(axis=0))


>>>>201204 finished!<<<<
>>>>201207 finished!<<<<
>>>>201210 finished!<<<<
>>>>201301 finished!<<<<
>>>>201304 finished!<<<<
>>>>201307 finished!<<<<
>>>>201310 finished!<<<<
>>>>201401 finished!<<<<
>>>>201404 finished!<<<<
>>>>201407 finished!<<<<
>>>>201410 finished!<<<<
>>>>201501 finished!<<<<
>>>>201504 finished!<<<<
>>>>201507 finished!<<<<
>>>>201510 finished!<<<<
>>>>201601 finished!<<<<
>>>>201604 finished!<<<<
>>>>201607 finished!<<<<
>>>> Feature generation complete! <<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 21ms/step - loss: 0.0022 - mean_absolute_percentage_error: 4.8588 - mape: 0.0486 - r: 0.9158 - theil_u: 0.0267
>>>>DJIA index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 16ms/step - loss: 0.0068 - mean_absolute_percentage_error: 6.4458 - mape: 0.0645 - r: 0.7349 - theil_u: 0.0363
>>>>DJIA index Data 201101 done!<<<<
No existing model, start to train!


Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.0032 - mean_absolute_percentage_error: 24.1488 - mape: 0.2396 - r: 0.9144 - theil_u: 0.1013
>>>>Nikkei 225 index Data 201204 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 23ms/step - loss: 0.0062 - mean_absolute_percentage_error: 33.5905 - mape: 0.3359 - r: 0.8600 - theil_u: 0.1939
>>>>Nikkei 225 index Data 201207 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 17ms/step - loss: 0.0047 - mean_absolute_percentage_error: 13.7212 - mape: 0.1372 - r: 0.9410 - theil_u: 0.0812
>>>>Nikkei 225 index Data 201210 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 14ms/step - loss: 0.1348 - mean_absolute_percentage_error: 23.3912 - mape: 0.2339 - r: 0.7171 - theil_u: 0.1690
>>>>Nikkei 225 index Data 201301 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==

Model Saved!
1/1 [==============================] - 0s 27ms/step - loss: 0.0086 - mean_absolute_percentage_error: 6.3880 - mape: 0.0639 - r: 0.9200 - theil_u: 0.0456
>>>>Nifty 50 index Data 201401 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.0175 - mean_absolute_percentage_error: 7.2903 - mape: 0.0729 - r: 0.9575 - theil_u: 0.0478
>>>>Nifty 50 index Data 201404 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.3638 - mean_absolute_percentage_error: 37.2374 - mape: 0.3724 - r: 0.0482 - theil_u: 0.2336
>>>>Nifty 50 index Data 201407 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 18ms/step - loss: 0.0481 - mean_absolute_percentage_error: 16.5097 - mape: 0.1651 - r: 0.5324 - theil_u: 0.0973
>>>>Nifty 50 index Data 201410 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [============

### SAE-LSTM

In [ ]:
result_dict = dict()
EPOCHS = 500 #0
past_history = 4

for index in INDEX_SHEET_NAME:
    print(f"Start {index} part!")
    result_dict[index] = dict()

    data_dir = f'../data/processed/sae/{index}'
    if not os.path.exists(data_dir):
        raw = load_data(sheet_name=index)
        generate_features(raw, index)
    
    train_lst = os.listdir(data_dir)
    train_lst.sort()
    for name in train_lst:
        x_train = np.load(data_dir + f'/{name}/X_train.npy')
        y_train = np.load(data_dir + f'/{name}/Y_train.npy')
        x_val = np.load(data_dir + f'/{name}/X_val.npy')
        y_val = np.load(data_dir + f'/{name}/Y_val.npy')
        x_test = np.load(data_dir + f'/{name}/X_test.npy')
        y_test = np.load(data_dir + f'/{name}/Y_test.npy')

        train_data, val_data, test_data = generate_train_val_data(
            x_train, y_train, x_val, y_val, x_test, y_test,
            past_history=4, batch_size=60
        )


        model_save_dir = f'../models/{index}/{name}'
        config_filename = '/sae-lstm_config.json'
        weight_filename = '/sae-lstm_weights.h5'

        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        if not os.path.exists(model_save_dir+config_filename) \
            or not os.path.exists(model_save_dir+weight_filename):
            print("No existing model, start to train!")
            lstm = build_lstm_model(inputs_shape=[4, 10],
                                    layers=5,
                                    units=[64, 64, 64, 64, 64],
                                    learning_rate=0.05)
            lstm.fit(train_data,
                     epochs=EPOCHS,
                     steps_per_epoch=(y_train.shape[0] // 60),
                     validation_data=val_data,
                     validation_steps=1,
                     verbose=0)
            json_config = lstm.to_json()

            with open(model_save_dir+config_filename, 'w') as json_file:
                json_file.write(json_config)
            # Save weights to disk
            lstm.save_weights(model_save_dir+weight_filename)
            print("Model Saved!")
        else:
            with open(model_save_dir+config_filename) as json_file:
                json_config = json_file.read()
            lstm = tf.keras.models.model_from_json(json_config)
            lstm.compile(loss='mse',
                              optimizer='Adam',
                              metrics=[
                                  tf.keras.metrics.MeanAbsolutePercentageError(),
                                  MeanAbsolutePercentageError(),
                                  LinearCorrelation(),
                                  TheilU()],
                              lr=learning_rate
                              )
            lstm.load_weights(model_save_dir+weight_filename)

            print("Model Loaded!")

        result_dict[index][name] = lstm.evaluate(test_data, steps=1)

        print(f">>>>{index} {name} done!<<<<")
    try:
        with open(f'./sae-lstm/{index}_train_result.pickle', 'wb') as handle:
            pickle.dump(result_dict[index], handle, protocol=pickle.HIGHEST_PROTOCOL)
    except:
        print('fail to save!')

with open(f'./sae-lstm/train_result.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

Start HangSeng Index Data part!
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 16ms/step - loss: 0.0072 - mean_absolute_percentage_error: 7.5702 - mape: 0.0757 - r: 0.8311 - theil_u: 0.0429
>>>>HangSeng Index Data 201010 done!<<<<
No existing model, start to train!
Model Saved!
1/1 [==============================] - 0s 14ms/step - loss: 0.0150 - mean_absolute_percentage_error: 11.2553 - mape: 0.1126 - r: 0.6324 - theil_u: 0.0627
>>>>HangSeng Index Data 201101 done!<<<<
No existing model, start to train!
